# Imports

In [15]:
import json
import pandas as pd
import matplotlib as mpl
from matplotlib import pyplot as plt
from sklearn import preprocessing as preproc
from sklearn.decomposition import PCA
import numpy as np

from sklearn.metrics import mean_squared_error

# import tqdm to notebook
from tqdm.notebook import tqdm

# https://stackoverflow.com/questions/52285104/3d-scatterplots-with-hue-colormap-and-legend
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns

try:
  from google.colab import drive
  drive.mount('/content/drive')

  COLAB = True
except ModuleNotFoundError:
  COLAB = False

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from minisom import MiniSom

import tensorflow as tf
from tensorflow import keras


import matplotlib.pyplot as plt
from matplotlib.patches import RegularPolygon, Ellipse
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import cm, colorbar
from matplotlib.lines import Line2D

# forcing tensorflow to use CPU

from datetime import datetime

%matplotlib inline

# Load Data

In [16]:
df_raw = pd.read_parquet("diabetic_data_cleaned_pca_data_enriched_som.parquet")

# fix some types that parquet didn't get right
df_raw["admission_type_id"] = df_raw["admission_type_id"].astype("category")
df_raw["discharge_disposition_id"] = df_raw["discharge_disposition_id"].astype("category")
df_raw["admission_source_id"] = df_raw["admission_source_id"].astype("category")

display(df_raw.shape)
display(df_raw.dtypes)
display(df_raw.head(2).T)

(100244, 43)

encounter_id                   int64
patient_nbr                    int64
race                        category
gender                      category
age                         category
admission_type_id           category
discharge_disposition_id    category
admission_source_id         category
time_in_hospital               int64
num_lab_procedures             int64
num_procedures                 int64
num_medications                int64
number_outpatient              int64
number_emergency               int64
number_inpatient               int64
diag_1                      category
diag_2                      category
diag_3                      category
number_diagnoses               int64
max_glu_serum               category
A1Cresult                   category
metformin                   category
repaglinide                 category
glimepiride                 category
glipizide                   category
glyburide                   category
pioglitazone                category
r

,0,1
encounter_id,149190,64410
patient_nbr,55629189,86047875
race,Caucasian,AfricanAmerican
gender,Female,Female
age,[10-20),[20-30)
admission_type_id,1,1
discharge_disposition_id,1,1
admission_source_id,7,7
time_in_hospital,3,2
num_lab_procedures,59,11


In [17]:
with open("data_dictionary.json", "r") as f:
  data_info = json.load(f)

# NUM_COLUMNS = data_info["Numeric Columns"]
NOM_COLUMNS = data_info["Nominal Columns"]
ORD_COLUMNS = data_info["Ordinal Columns"]
TGT = data_info["Target"]

# Auto Encoders

In [18]:
df_temp = df_raw.copy()

CAT_COLUMNS = NOM_COLUMNS + ORD_COLUMNS

df_temp = df_temp[CAT_COLUMNS]
for col in CAT_COLUMNS:
  df_temp[col] = df_temp[col].astype("category")

ohe = OneHotEncoder(drop=None, sparse_output=False)
temp = ohe.fit_transform(df_temp)

temp.shape

(100244, 2363)

In [19]:
strat = df_raw[[TGT]]
X_train, X_test, _, _ = train_test_split(temp, strat, test_size=0.2, stratify=strat, random_state=42)

X_train.shape, X_test.shape

((80195, 2363), (20049, 2363))

In [20]:
def train_autoenc(train_data, df_ref, latent_vars, learning_rate, epochs):
    input_shape = (2363,)

    inp = keras.Input(shape=input_shape)

    enc = keras.layers.Dense(400, activation="relu")(inp)
    enc = keras.layers.Dense(150, activation="relu")(enc)
    enc = keras.layers.Dense(latent_vars, activation="relu", name="encoder_out")(enc)

    dec = keras.layers.Dense(latent_vars, activation="relu")(enc)
    dec = keras.layers.Dense(150, activation="relu")(dec)
    dec = keras.layers.Dense(400, activation="relu")(dec)

    outputs = []
    outs = 0

    # create output layers
    for c in df_ref:
        out_num = df_ref[c].nunique()
        if df_ref[c].isna().sum() > 0:
            out_num += 1

        out_temp = keras.layers.Dense(out_num, activation="softmax", name=f"out_{c}")(dec)
        outs += out_num
        outputs.append(out_temp)

    auto_encoder = keras.Model(inp, outputs)
    auto_encoder.summary()
    
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        learning_rate,
        decay_steps=1500,  # 1 epoch = 1254 steps
        decay_rate=0.98,  # reduce to 10% in each +- 120 epochs
        staircase=True)  # If True, learning rate decays in a discrete staircase, else smoothly

    opt = keras.optimizers.Adam(learning_rate=lr_schedule)

    # opt = keras.optimizers.Adam(learning_rate=learning_rate)

    losses = {f"out_{c}": "categorical_crossentropy" for c in df_ref}
    metrics = {f"out_{c}": ["accuracy"] for c in df_ref}

    auto_encoder.compile(optimizer=opt, loss=losses, metrics=metrics)
    run_id = datetime.now().strftime("%Y%m%d-%H%M%S")
    log_dir = "logs/fit/" + run_id

    print(log_dir)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    targets = []

    idx = 0
    for o in outputs:
        tgt = train_data[:, idx:idx+o.shape[1]]
        targets.append(tgt)
        idx += o.shape[1]
        
    hist = auto_encoder.fit(train_data, targets,
                            epochs=epochs, batch_size=64, shuffle=True,
                            callbacks=[tensorboard_callback],
                            verbose=2)

    with open(f"auto_encoder_{run_id}.json", "w") as f:
        d = hist.history
        d["run_id"] = run_id
        d["latent_variables"] = latent_vars
        d["learning_rate"] = learning_rate    
        d["epochs"] = epochs    
        json.dump(d, f, indent=4)

        # save weights
        auto_encoder.save(f"auto_encoder_{run_id}.h5")
    
    return auto_encoder


In [21]:
tf.random.set_seed(42)
np.random.seed(42)

ae = train_autoenc(X_train, df_temp, latent_vars=20, learning_rate=0.001, epochs=200)

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 2363)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 400)       │    945,600 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 150)       │     60,150 │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encoder_out (Dense) │ (None, 20)        │      3,020 │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 20)        │        420 │ encoder_out[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 150)       │      3,150 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 400)       │     60,400 │ dense_13[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_race (Dense)    │ (None, 5)         │      2,005 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_gender (Dense)  │ (None, 2)         │        802 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_diag_1 (Dense)  │ (None, 715)       │    286,715 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_diag_2 (Dense)  │ (None, 743)       │    297,943 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_diag_3 (Dense)  │ (None, 789)       │    316,389 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_change (Dense)  │ (None, 2)         │        802 │ dense_14[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_diabetesMed     │ (None, 2)         │        802 │ dense_14[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_admission_type… │ (None, 8)         │      3,208 │ dense_14[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_discharge_disp… │ (None, 26)        │     10,426 │ dense_14[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_admission_sour… │ (None, 17)        │      6,817 │ dense_14[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_A1Cresult       │ (None, 4)         │      1,604 │ dense_14[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_max_glu_serum   │ (None, 4)         │      1,604 │ dense_14[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_metformin       │ (None, 4)         │      1,604 │ dense_14[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ out_repaglinide     │ (None, 4)         │      1,604 │ dense_14[0][0]  

 Total params: 2,020,303 (7.71 MB)

 Trainable params: 2,020,303 (7.71 MB)

 Non-trainable params: 0 (0.00 B)

logs/fit/20240402-021303
Epoch 1/200
1254/1254 - 12s - 10ms/step - loss: 16.9863 - out_A1Cresult_accuracy: 0.8566 - out_admission_source_id_accuracy: 0.8550 - out_admission_type_id_accuracy: 0.8504 - out_age_accuracy: 0.6894 - out_change_accuracy: 0.9722 - out_diabetesMed_accuracy: 0.9832 - out_diag_1_accuracy: 0.2071 - out_diag_2_accuracy: 0.2069 - out_diag_3_accuracy: 0.1889 - out_discharge_disposition_id_accuracy: 0.7674 - out_gender_accuracy: 0.9267 - out_glimepiride_accuracy: 0.9494 - out_glipizide_accuracy: 0.9437 - out_glyburide-metformin_accuracy: 0.9924 - out_glyburide_accuracy: 0.9445 - out_insulin_accuracy: 0.8738 - out_max_glu_serum_accuracy: 0.9541 - out_metformin_accuracy: 0.9388 - out_pioglitazone_accuracy: 0.9524 - out_race_accuracy: 0.9161 - out_repaglinide_accuracy: 0.9817 - out_rosiglitazone_accuracy: 0.9515
Epoch 2/200
1254/1254 - 4s - 3ms/step - loss: 11.3679 - out_A1Cresult_accuracy: 0.8946 - out_admission_source_id_accuracy: 0.9016 - out_admission_type_id_accurac

In [22]:
preds = ae.predict(X_train)

concat_preds = np.concatenate(preds, axis=1)
mse = mean_squared_error(X_train, concat_preds)
print(f"MSE Train: {mse:2.4f}")

lower = 0
outs = [o.shape[1] for o in ae.outputs]
for i, upper in enumerate(outs):
    pred = preds[i]
    print(f"Prediction {ohe.feature_names_in_[i]}: {pred.shape}", end=" ")

    acc = 0
    for j in range(len(preds[i])):
        acc += np.argmax(X_train[j, lower:lower+upper]) == np.argmax(pred[j, 0:upper])
    acc /= len(pred)
    print(f"Accuracy: {acc:2.2%}")
    lower += upper

I0000 00:00:1712021066.096018  168130 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_66', 28 bytes spill stores, 28 bytes spill loads



2507/2507 ━━━━━━━━━━━━━━━━━━━━ 4s 944us/step
MSE Train: 0.0000
Prediction race: (80195, 5) Accuracy: 100.00%
Prediction gender: (80195, 2) Accuracy: 100.00%
Prediction diag_1: (80195, 715) Accuracy: 100.00%
Prediction diag_2: (80195, 743) Accuracy: 100.00%
Prediction diag_3: (80195, 789) Accuracy: 100.00%
Prediction change: (80195, 2) Accuracy: 100.00%
Prediction diabetesMed: (80195, 2) Accuracy: 100.00%
Prediction admission_type_id: (80195, 8) Accuracy: 100.00%
Prediction discharge_disposition_id: (80195, 26) Accuracy: 100.00%
Prediction admission_source_id: (80195, 17) Accuracy: 100.00%
Prediction A1Cresult: (80195, 4) Accuracy: 100.00%
Prediction max_glu_serum: (80195, 4) Accuracy: 100.00%
Prediction metformin: (80195, 4) Accuracy: 100.00%
Prediction repaglinide: (80195, 4) Accuracy: 100.00%
Prediction glimepiride: (80195, 4) Accuracy: 100.00%
Prediction glipizide: (80195, 4) Accuracy: 100.00%
Prediction glyburide: (80195, 4) Accuracy: 100.00%
Prediction pioglitazone: (80195, 4) Acc

In [23]:
preds = ae.predict(X_test)

concat_preds = np.concatenate(preds, axis=1)
mse = mean_squared_error(X_test, concat_preds)
print(f"MSE Test: {mse:2.4f}")

lower = 0
outs = [o.shape[1] for o in ae.outputs]
for i, upper in enumerate(outs):
    pred = preds[i]
    print(f"Prediction {ohe.feature_names_in_[i]}: {pred.shape}", end=" ")

    acc = 0
    for j in range(len(preds[i])):
        acc += np.argmax(X_test[j, lower:lower+upper]) == np.argmax(pred[j, 0:upper])
    acc /= len(pred)
    print(f"Accuracy: {acc:2.2%}")
    lower += upper

570/627 ━━━━━━━━━━━━━━━━━━━━ 0s 619us/step

I0000 00:00:1712021079.361297  168420 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_20', 20 bytes spill stores, 20 bytes spill loads



627/627 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step
MSE Test: 0.0008
Prediction race: (20049, 5) Accuracy: 98.79%
Prediction gender: (20049, 2) Accuracy: 99.93%
Prediction diag_1: (20049, 715) Accuracy: 72.51%
Prediction diag_2: (20049, 743) Accuracy: 76.64%
Prediction diag_3: (20049, 789) Accuracy: 73.28%
Prediction change: (20049, 2) Accuracy: 99.56%
Prediction diabetesMed: (20049, 2) Accuracy: 99.83%
Prediction admission_type_id: (20049, 8) Accuracy: 98.92%
Prediction discharge_disposition_id: (20049, 26) Accuracy: 96.23%
Prediction admission_source_id: (20049, 17) Accuracy: 96.18%
Prediction A1Cresult: (20049, 4) Accuracy: 98.99%
Prediction max_glu_serum: (20049, 4) Accuracy: 99.15%
Prediction metformin: (20049, 4) Accuracy: 99.05%
Prediction repaglinide: (20049, 4) Accuracy: 98.75%
Prediction glimepiride: (20049, 4) Accuracy: 98.57%
Prediction glipizide: (20049, 4) Accuracy: 99.24%
Prediction glyburide: (20049, 4) Accuracy: 99.09%
Prediction pioglitazone: (20049, 4) Accuracy: 99.09%
Predicti

In [24]:
encoder = keras.Model(inputs=ae.input, outputs=ae.get_layer("encoder_out").output)

In [25]:
ae_vars = encoder.predict(temp)
df_ae = pd.DataFrame(ae_vars, columns=[f"ae_temp_{i}" for i in range(ae_vars.shape[1])])

final_vars = []
for c in df_ae:
    if df_ae[c].max() - df_ae[c].min() == 0:
        continue

    final_vars.append(c)

df_ae = df_ae[final_vars].rename(columns={c: f"ae_{i}" for i, c in enumerate(final_vars)})

3133/3133 ━━━━━━━━━━━━━━━━━━━━ 2s 560us/step


In [26]:
df_final = df_raw.join(df_ae)
display(df_final.describe().T)

filename = "diabetic_data_cleaned_pca_som_diag_grouped_ae"

df_final.to_csv(f"{filename}.csv", index=False)
df_final.to_parquet(f"{filename}.parquet")

,count,mean,std,min,25%,50%,75%,max
encounter_id,100244.0,1.660172e+08,1.026756e+08,12522.000000,8.556777e+07,1.530494e+08,2.310625e+08,4.438672e+08
patient_nbr,100244.0,5.462971e+07,3.869452e+07,135.000000,2.346104e+07,4.604908e+07,8.778530e+07,1.895026e+08
time_in_hospital,100244.0,4.420275e+00,2.991273e+00,1.000000,2.000000e+00,4.000000e+00,6.000000e+00,1.400000e+01
num_lab_procedures,100244.0,4.317372e+01,1.969148e+01,1.000000,3.200000e+01,4.400000e+01,5.700000e+01,1.320000e+02
num_procedures,100244.0,1.350275e+00,1.710913e+00,0.000000,0.000000e+00,1.000000e+00,2.000000e+00,6.000000e+00
num_medications,100244.0,1.611782e+01,8.116424e+00,1.000000,1.100000e+01,1.500000e+01,2.000000e+01,8.100000e+01
number_outpatient,100244.0,3.725610e-01,1.274074e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,4.200000e+01
number_emergency,100244.0,1.990144e-01,9.335410e-01,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,7.600000e+01
number_inpatient,100244.0,6.394697e-01,1.262773e+00,0.000000,0.000000e+00,0.000000e+00,1.000000e+00,2.100000e+01
number_diagnoses,100244.0,7.497007e+00,1.839537e+00,3.000000,6.000000e+00,8.000000e+00,9.000000e+00,1.600000e+01


## Conclusion

This notebook finishes adding 11 new columns. The auto-encoder model could replace the 22 original categorical variables (with more than 2400 combined levels) with 11 numerical variables. It is an unsupervised learning technique since the goal is to recreate the original data (without the target!) with only these 11 numerical variables. It achieved an excellent result. This set of 11 variables can be used in place of the original categorical variables if you want (the same for SOM_JAC and SOM_EUC).